In [17]:
from sentence_transformers import SentenceTransformer, losses, InputExample, models, util, quantization
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from torch.utils.data import DataLoader
import torch
from torch import nn

import os

import pandas as pd
import numpy as np

from custom_adapter_module import AdapterModule

import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/home/estudiante/mlt_project/utils/__init__.py'>

In [2]:
# Carga del modelo de embeddings de palabras
word_embedding_model = models.Transformer(
    model_name_or_path="sentence-transformers/all-MiniLM-L12-v2",  # Modelo base de Sentence Transformers
    max_seq_length=128,  # Longitud máxima de la secuencia
    do_lower_case=False  # No convertir a minúsculas
)

# Definición de los parámetros del modelo de pooling
pooling_model = models.Pooling(
    word_embedding_dimension=384,  # Dimensión de los embeddings de palabras
    pooling_mode_cls_token=False,  # No usar el token CLS para el pooling
    pooling_mode_mean_tokens=True,  # Usar el promedio de los tokens para el pooling
    pooling_mode_max_tokens=False,  # No usar el máximo de los tokens para el pooling
    pooling_mode_mean_sqrt_len_tokens=False,  # No usar el promedio de la raíz cuadrada de la longitud para el pooling
    pooling_mode_weightedmean_tokens=False,  # No usar el promedio ponderado de los tokens para el pooling
    pooling_mode_lasttoken=False,  # No usar el último token para el pooling
    include_prompt=True  # Incluir el prompt en el pooling
)

# Definición del modelo de normalización
normalize = models.Normalize()

# Congelar los pesos del modelo de embeddings de palabras para que no se entrenen
for param in word_embedding_model.parameters():
    param.requires_grad = False

# Configuración del dispositivo para usar GPU si está disponible, de lo contrario usar CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definir el módulo adaptador con las dimensiones de entrada y salida
adapter = AdapterModule(384, 384).to(device)

# Definir el modelo base de Sentence Transformer con las capas de embedding, pooling y normalización
base_model = SentenceTransformer(modules=[word_embedding_model, pooling_model, normalize], device=device)

# Definir el modelo personalizado de Sentence Transformer que incluye el adaptador
custom_domain_model = SentenceTransformer(
    modules=[word_embedding_model, pooling_model, adapter, normalize], device=device
)

custom_domain_model  # Mostrar la arquitectura del modelo personalizado

/home/estudiante/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): AdapterModule(
    (dense1): Linear(in_features=384, out_features=640, bias=True)
    (dense2): Linear(in_features=640, out_features=512, bias=True)
    (output): Linear(in_features=512, out_features=384, bias=True)
    (activation): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (3): Normalize()
)

In [3]:
qa = pd.read_pickle('data/qa_training.pkl')
qa_eval = pd.read_pickle('data/qa_evaluation.pkl')

In [4]:
train_examples = [
    InputExample(texts=[qa[0], qa[1]])
    for qa in qa
]

In [5]:
print("Training lenght: ", len(qa))
print("Validation lenght: ", len(qa_eval['queries']))

Training lenght:  12105
Validation lenght:  1216


In [6]:
# Crear el dataset de entrenamiento
# Se crea una lista de InputExample, donde cada ejemplo es un par de textos (pregunta y respuesta)
train_dataset = [
    InputExample(texts=[qa[0], qa[1]])
    for qa in qa
]

# Crear el DataLoader para el dataset de entrenamiento
# shuffle=True permite mezclar los datos en cada época
loader = DataLoader(train_dataset, shuffle=True, batch_size=256)

# Definir la función de pérdida
# MultipleNegativesSymmetricRankingLoss es adecuada para tareas de recuperación de información con pares positivos
train_loss = losses.MultipleNegativesSymmetricRankingLoss(custom_domain_model)

# Definir el evaluador
# InformationRetrievalEvaluator evalúa el modelo en un conjunto de consultas y corpus
evaluator = InformationRetrievalEvaluator(
    qa_eval['queries'], 
    qa_eval['corpus'], 
    qa_eval['relevant_docs'], 
    name='qa_eval', 
    main_score_function='dot_score'
)

# Definir el número de épocas y los pasos de calentamiento
epochs = 500
warmup_steps = int(len(loader) * epochs * 0.1)

# Entrenar el modelo
# fit() entrena el modelo con los objetivos de entrenamiento y evaluador
custom_domain_model.fit(
    train_objectives=[(loader, train_loss)],  # Objetivos de entrenamiento: DataLoader y función de pérdida
    epochs=epochs,  # Número de épocas
    warmup_steps=warmup_steps,  # Número de pasos de calentamiento
    output_path='results/domain_adaptation_model',  # Ruta de salida para guardar el modelo entrenado
    show_progress_bar=True,  # Mostrar barra de progreso durante el entrenamiento
    save_best_model=True,  # Guardar el mejor modelo según la evaluación
    use_amp=True,  # Habilitar Mixed Precision 
    evaluator=evaluator,  # Evaluador para evaluar el modelo durante el entrenamiento
    evaluation_steps=50,  # Evaluar el modelo cada 50 pasos
)


Epoch: 100%|███████████████████████████████████████████████████████████████████████████████████| 500/500 [31:51<00:00,  3.82s/it]


## Evaluating the base model & the custom model

In [7]:
hit_rate = utils.hit_rate_at_k(qa_eval['queries'], qa_eval['corpus'], qa_eval['relevant_docs'], k=10, model=base_model)
print(f"Hit Rate @ 10 (base model): {round(hit_rate, 2)}")


hit_rate = utils.hit_rate_at_k(qa_eval['queries'], qa_eval['corpus'], qa_eval['relevant_docs'], k=10, model=custom_domain_model)
print(f"Hit Rate @ 10 (custom model): {round(hit_rate, 2)}")

Hit Rate @ 10 (base model): 0.1
Hit Rate @ 10 (custom model): 0.87


In [8]:
eva_base_model = evaluator(base_model, output_path='results/base_model/')
print("MAP @ 100 (base model): ", round(eva_base_model, 3))

eva_custom_model = evaluator(custom_domain_model, output_path='results/custom_model/')
print("MAP @ 100 (custom model): ", round(eva_custom_model, 3))

MAP @ 100 (base model):  0.002
MAP @ 100 (custom model):  0.026


In [14]:
base_model_eval = pd.read_csv('results/base_model/Information-Retrieval_evaluation_qa_eval_results.csv')
base_model_eval['tipo'] = 'base_model'
custom_model_eval = pd.read_csv('results/custom_model/Information-Retrieval_evaluation_qa_eval_results.csv')
custom_model_eval['tipo'] = 'custom_model'

pd.concat([base_model_eval, custom_model_eval]).to_csv('results/eval_comparation.csv', index=False)

pd.concat([base_model_eval, custom_model_eval])


,epoch,steps,cos_sim-Accuracy@1,cos_sim-Accuracy@3,cos_sim-Accuracy@5,cos_sim-Accuracy@10,cos_sim-Precision@1,cos_sim-Recall@1,cos_sim-Precision@3,cos_sim-Recall@3,...,dot_score-Precision@3,dot_score-Recall@3,dot_score-Precision@5,dot_score-Recall@5,dot_score-Precision@10,dot_score-Recall@10,dot_score-MRR@10,dot_score-NDCG@10,dot_score-MAP@100,tipo
0,-1,-1,0.034539,0.055099,0.069901,0.104441,0.034539,0.000934,0.018366,0.001448,...,0.018366,0.001448,0.013980,0.001823,0.010444,0.003206,0.051652,0.014119,0.001808,base_model
0,-1,-1,0.347039,0.625000,0.729441,0.866776,0.347039,0.016560,0.208333,0.032509,...,0.208333,0.032509,0.145888,0.038286,0.086678,0.046927,0.512151,0.136737,0.026438,custom_model


### El modelo sigue preservando las propiedades del conocimiento base

In [10]:
paper = "Composable Lightweight Processors"
author = "Changkyu Kim"
author2 = "Venturelli Guilherme Cavalheiro"

concept1 = "shark"
concept2 = "ocean"
concept3 = "strawberry"

In [11]:
custom_paper = custom_domain_model.encode(paper)
custom_author = custom_domain_model.encode(author)
custom_author2 = custom_domain_model.encode(author2)

custom_concept1 = custom_domain_model.encode(concept1)
custom_concept2 = custom_domain_model.encode(concept2)
custom_concept3 = custom_domain_model.encode(concept3)

# Imprimir los resultados y explicaciones
print(f"Producto punto entre dos conceptos (shark y ocean): {np.dot(custom_concept1, custom_concept2)}")
print(f"Producto punto entre dos conceptos (shark y strawberry): {np.dot(custom_concept1, custom_concept3)}")
print(f"Producto punto entre el documento y un concepto (ocean): {np.dot(custom_paper, custom_concept2)}")

Producto punto entre dos conceptos (shark y ocean): 0.6271718740463257
Producto punto entre dos conceptos (shark y strawberry): 0.4619179368019104
Producto punto entre el documento y un concepto (ocean): -0.07007230073213577


In [12]:
base_paper = base_model.encode(paper)
base_author = base_model.encode(author)
base_author2 = base_model.encode(author2)

base_concept1 = base_model.encode(concept1)
base_concept2 = base_model.encode(concept2)
base_concept3 = base_model.encode(concept3)  

# Imprimir los resultados y explicaciones
print(f"Producto punto entre dos conceptos (shark y ocean): {np.dot(base_concept1, base_concept2)}")
print(f"Producto punto entre dos conceptos (shark y strawberry): {np.dot(base_concept1, base_concept3)}")
print(f"Producto punto entre el documento y un concepto (ocean): {np.dot(base_paper, base_concept2)}")

Producto punto entre dos conceptos (shark y ocean): 0.5232284069061279
Producto punto entre dos conceptos (shark y strawberry): 0.23324596881866455
Producto punto entre el documento y un concepto (ocean): -0.08226228505373001
